<a href="https://colab.research.google.com/github/d-vinha/SPBD/blob/main/lab2/SPBD_Labs_mapreduce2_exercise.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Python MapReduce Exercises

##1. MrJob MapReduce Word Frequency

Using the [MrJob](https://mrjob.readthedocs.io/en/latest/) library, create a map-reduce program that counts the number of occurrences of each word, **sorting** them by frequency (the words with higher occurrence first).

Check the MrJob documentation to see how multi-step MapReduce jobs can
be implemented in the same Python class.

Note that you will need a notebook cell to install MrJob before you run your solution, as shown in this week's example notebook.

In [1]:
#@title Download the dataset and install MrJob
!wget -q -O os_maias.txt https://www.dropbox.com/s/n24v0z7y79np319/os_maias.txt?dl=0
!pip install mrjob --quiet
!wget -q -O /etc/mrjob.conf https://raw.githubusercontent.com/smduarte/spbd-2324/main/lab2/mrjob.conf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 439.6/439.6 kB 5.9 MB/s eta 0:00:00


### MrJob Word Frequency 1st Mapper-Reducer
Read the words from input and count each ***unique*** word.

The processing is split into:

+ The mapper emits for each line the number of words
+ The reducer sums all the tuples produced by the mapper stage and emit a list of tuples that each unique word and its count

Using MrJob, a MapReduce job can be expressed in a single Python class,
with methods for each of the phases. The reducer phase is called separately for each key, with the collection of values to be reduced.



In [6]:
%%file eachwordcount.py

from mrjob.job import MRJob, MRStep
import string

max_freq=100000

class MRWordCountFrequency(MRJob):

    def mapper_words(self, _, line):
      line = line.strip()
      line = line.translate(str.maketrans('', '', string.punctuation+'«»'))
      for word in line.split():
        yield word, 1
      # get the raw results

    def reducer_words(self, key, values):
        yield key, sum(values)
      # generates tuples with words and the sum of their frequencies

    def mapper_partition_sort(self, word, freq):
      yield '%05d' % (max_freq-freq), word
      # to solve the problem that 11 < 2 and other (because MR uses alphabetical
      # sorting of strings) we insert leading 00s; to solve the problem of this
      # order being descending, we invert the frequencies of each word by subtra
      # it to a high number

    def reducer_partition_sort(self, freq, words):
      for word in words:
        yield word, max_freq-int(freq)
      # reverse the tuple again and with the actual frequency
      # this will be executed in a partition fashion across multiple machines
      # this reducer guarantees the partitions are sorted but there is no final/
      # general order

    def mapper_total_sort(self, word, freq):
      yield None, (word, freq)
      # to solve the previous and get the final order - funnel every result to a
      # single key (None in this case - a constant in Python for when the key
      # does not matter)

    def reducer_total_sort(self, _, values):
      for word, freq in sorted(values, key= lambda x: x[1], reverse=True):
        yield word, freq
      # This is only possible because since there is only 1 key (None), this
      # will be done in a single machine

    def steps(self):
        return [ MRStep(mapper=self.mapper_words, reducer=self.reducer_words),
                 MRStep(mapper=self.mapper_partition_sort,
                        reducer=self.reducer_partition_sort),
                 MRStep(mapper=self.mapper_total_sort,
                        reducer=self.reducer_total_sort)]

if __name__ == '__main__':
    MRWordCountFrequency.run()

Overwriting eachwordcount.py


In [7]:
!rm -rf results
!python -m eachwordcount  --output-dir results --cleanup NONE os_maias.txt
!head results/*

Using configs in /etc/mrjob.conf
Running step 1 of 3...
Creating temp directory /tmp/eachwordcount.root.20230928.102216.385278
Running step 2 of 3...
Running step 3 of 3...
job output is in results
"de"	8311
"a"	6736
"o"	6615
"que"	4986
"e"	4533
"um"	3026
"com"	2794
"do"	2571
"da"	2202
"uma"	2170


##2. Weblog Analysis

Consider a set of log files captured during a DDOS (*Distributed Denial of Service*) attack, containing information for the web accesses performed during the attack to the server.

Create a new notebook that processes the log of web entries using MrJob and map-reduce to:

1. Count the number of unique IP addresses involved in the DDOS attack.

2. For each interval of 10 seconds, provide the following information: [number of requests, average execution time, maximum time, minimum time]

3. Create an inverted index that, for each interval of 10 seconds, has a list of (unique) IPs executing accesses (to each URL).


The log files contain text lines as shown below, with TAB as the separator:

date |IP_source | status_code | operation | URL | execution time |
-|-|-|-|-|-
timestamp  | string | int | string | string| float |
2016-12-06T08:58:35.318+0000|37.139.9.11|404|GET|/codemove/TTCENCUFMH3C|0.026

<br>
The log can be downloaded from:

[https://www.dropbox.com/s/0r8902uj9yum7dg/web.log?dl=0](https://www.dropbox.com/s/0r8902uj9yum7dg/web.log?dl=0)

Suggestion: to start, make a copy an existing notebook and modify it.

If you really must..., you can use [dateutil.parser](https://dateutil.readthedocs.io/en/stable/parser.html) for decoding timestamps.

In [8]:
!wget -q -O weblog https://www.dropbox.com/s/0r8902uj9yum7dg/web.log?dl=0
!wc weblog

  143457   860742 11758533 weblog


### 1. Count the number of unique IP addresses involved in the DDOS attack.

In [9]:
%%file unique_ips.py

from mrjob.job import MRJob, MRStep

class MRUniqueIPs(MRJob):

    def mapper_ip(self, _, line):
      _, ip_source, _ = line.strip().split(' ', 2)
      yield ip_source, None
      # We output the ip_source as the key (value is unimportant, so it is none)

    def reducer_ip(self, ip_source, _):
      yield None, 1
      # the reducer, for each key (ip_source) it is called only once with all
      # its repetition
      # because we are not interested in the IP itself but just the number of IP
      # we use none as the key and 1 as the value to indicate unique keys
      # (ip_sources in this case)

    def reducer_filter(self, _, ips):
      yield "Unique IPs:", sum(ips)
    # This is only possible because since there is only 1 key (None), this
    # will be done in a single machine

    def steps(self):
      return [MRStep(mapper=self.mapper_ip, reducer=self.reducer_ip),
              MRStep(reducer=self.reducer_filter)]

if __name__ == '__main__':
    MRUniqueIPs.run()

Writing unique_ips.py


In [10]:
!rm -rf results
!python -m unique_ips  --output-dir results --cleanup NONE weblog
!head results/*

Using configs in /etc/mrjob.conf
Running step 1 of 2...
Creating temp directory /tmp/unique_ips.root.20230928.103153.759604
Running step 2 of 2...
job output is in results
"Unique IPs:"	167


### 2. For each 10 sec interval, provide: [num requests, average exec time, max time, min time]

In [15]:
%%file interval_info.py

from statistics import *
from mrjob.job import MRJob, MRStep

class MRIntervalInfo(MRJob):

  def mapper(self, _, line):
        vals = line.strip().split(' ')
        timestamp = vals[0]
        #getting the date and time
        execution_time = float(vals[5])
        interval = timestamp[0:18] # YYYY-MM-DDTHH:MM:S -> 10s intervals
        #We use the string directely, truncated at the 10s mark (so ignoring the
        #last digit of secs)
        yield interval, execution_time

  def reducer(self, interval, values):
      times = list(values)
      # we funnel results of values to a list, transforming the generator
      # (values) because as a list it can be used to compute the times
      yield interval, (len(times), mean(times), max(times), min(times))

  def steps(self):
      return [MRStep(mapper=self.mapper, reducer=self.reducer)]

if __name__ == '__main__':
    MRIntervalInfo.run()

Overwriting interval_info.py


In [17]:
!rm -rf results
!python -m interval_info --output-dir results --cleanup NONE weblog
!head results/*

Using configs in /etc/mrjob.conf
Running step 1 of 1...
Creating temp directory /tmp/interval_info.root.20230928.103806.524450
job output is in results
==> results/part-00000 <==
"2016-12-06T08:58:3"	[483, 7.593424430641822, 46.849, 0.013]
"2016-12-06T08:58:4"	[2611, 30.15984565300651, 69.654, 0.014]
"2016-12-06T08:58:5"	[5500, 38.52511163636364, 80.846, 0.017]
"2016-12-06T08:59:0"	[6914, 38.534382123228234, 81.659, 0.018]
"2016-12-06T08:59:1"	[6271, 32.96384978472333, 83.993, 0.017]
"2016-12-06T08:59:2"	[5434, 17.29333143172617, 77.967, 0.051]
"2016-12-06T08:59:3"	[8015, 11.21015221459763, 67.441, 0.056]
"2016-12-06T08:59:4"	[7947, 7.7618157795394485, 65.706, 0.914]

==> results/part-00001 <==
"2016-12-06T08:59:5"	[5983, 3.8216643824168477, 54.29, 0.678]
"2016-12-06T09:00:0"	[6882, 8.649971519907004, 45.314, 0.017]
"2016-12-06T09:00:1"	[9719, 7.857372672085606, 34.406, 0.225]
"2016-12-06T09:00:2"	[6616, 4.611345223700121, 25.847, 0.014]
"2016-12-06T09:00:3"	[6771, 1.6047638458130262, 

### 3. Create an inverted index that, for each interval of 10 seconds, has a list of (unique) IPs executing accesses (to each URL).

In [ ]:
%%file inverted_index.py

from mrjob.job import MRJob, MRStep

class MRInvertedIndex(MRJob):

  def mapper(self, _, line):
        vals = line.strip().split(' ')
        if len(vals) >= 6:
          timestamp = vals[0]
          interval = timestamp[0:18] # YYYY-MM-DDTHH:MM:S -> 10s intervals

          source_ip = vals[1]
          target_url = vals[4]
          yield "{}-{}".format(interval, target_url), source_ip
          # my key is

  def reducer(self, key, values):
    yield key, set(values)
    # instead of a list, to remove repetitions, we use a set

if __name__ == '__main__':
    MRInvertedIndex.run()

In [ ]:
!rm -rf results
!python -m inverted_index --output-dir results web.log && head results/*